This code is used for automated cryptocurrency trading based on Bollinger Bands, a type of statistical chart characterizing the prices and volatility over time of a financial instrument or commodity.

Here’s how it works:

1. Initialize the Exchange: The code first sets up a connection to the WazirX exchange.
2. Fetch Prices: It then continuously fetches the latest prices of the specified cryptocurrency pair (in this case, “xmrusdt” which represents Monero to Tether).
3. Calculate Bollinger Bands: The code calculates the Bollinger Bands for the specified cryptocurrency pair. Bollinger Bands consist of a middle band being an N-period simple moving average (SMA), an upper band at K times an N-period standard deviation above the middle band, and a lower band at K times an N-period standard deviation below the middle band. The parameters N and K are typically 20 and 2, respectively.
4. Execute Trades: If the current price is higher than the upper band, it indicates that the cryptocurrency is overbought, and it might be a good time to sell. Conversely, if the current price is lower than the lower band, it indicates that the cryptocurrency is oversold, and it might be a good time to buy.
The financial principle behind this is that the prices tend to stay within the upper and lower bands. Therefore, when the price goes outside of these bands, it is considered unusual, and it is expected to revert back within the bands. This strategy is known as mean reversion.

However, it’s important to note that in real-world trading, factors such as transaction fees, market liquidity, and price slippage can affect the profitability of trading strategies. Also, using this kind of automated trading bot requires careful risk management and is not without risks. Always use caution and consult with a financial advisor before engaging in cryptocurrency trading.

Please replace "your-smtp-server.com", your-port, "your-email@example.com", "your-password", "<your-api-key>", and "<your-secret-key>" with your actual SMTP server details, email details, and WazirX API credentials respectively. The email functionality is used to send notifications when a buy or sell signal is generated. The rate limiter is used to ensure that the code does not exceed the API’s rate limit. The logging functionality is used to log information about when buy or sell signals are generated and when there are errors. The code runs indefinitely, checking for trading signals every 60 seconds.

In [ ]:
import requests
import time
import numpy as np
import pandas as pd
import logging
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from scipy.stats import norm
from ratelimiter import RateLimiter

# Define the API endpoint
base_url = "https://api.wazirx.com"

In [ ]:
# Set up logging
logging.basicConfig(filename='trading_bot.log', level=logging.INFO)


In [ ]:
# Set up rate limiter
rate_limiter = RateLimiter(max_calls=10, period=1)  # Adjust these values based on the API's rate limit

def send_email(subject, message):
    """Send an email notification."""
    # Set up the SMTP server
    s = smtplib.SMTP(host='your-smtp-server.com', port=your-port)
    s.starttls()
    s.login("your-email@example.com", "your-password")

    # Create the email
    msg = MIMEMultipart()
    msg['From'] = "your-email@example.com"
    msg['To'] = "your-email@example.com"
    msg['Subject'] = subject
    msg.attach(MIMEText(message))

    # Send the email
    s.send_message(msg)
    s.quit()

In [ ]:
@rate_limiter
def get_price(symbol):
    """Fetch the latest price of a symbol."""
    try:
        response = requests.get(f"{base_url}/api/v1/ticker/24hr", params={"symbol": symbol})
        data = response.json()
        return float(data['lastPrice'])
    except Exception as e:
        logging.error(f"Error fetching price: {e}")
        send_email("Trading Bot Error", f"Error fetching price: {e}")
        return None

In [ ]:
@rate_limiter
def calculate_bollinger_bands(symbol, period=20, std_dev_factor=2):
    """Calculate the Bollinger Bands for a symbol."""
    try:
        # Fetch historical data
        response = requests.get(f"{base_url}/api/v1/klines", params={"symbol": symbol, "interval": "1d", "limit": period})
        data = response.json()
        closing_prices = [float(x[4]) for x in data]  # 4th index is the closing price
        df = pd.DataFrame(closing_prices, columns=['Close'])

        # Calculate the SMA and standard deviation
        sma = df['Close'].rolling(window=period).mean()
        std_dev = df['Close'].rolling(window=period).std()

        # Calculate the upper and lower Bollinger Bands
        upper_band = sma + std_dev_factor * std_dev
        lower_band = sma - std_dev_factor * std_dev

        return upper_band.iloc[-1], lower_band.iloc[-1]
    except Exception as e:
        logging.error(f"Error calculating Bollinger Bands: {e}")
        send_email("Trading Bot Error", f"Error calculating Bollinger Bands: {e}")
        return None, None

In [ ]:
def trade(symbol, api_key, secret_key):
    """Main trading loop."""
    while True:
        upper_band, lower_band = calculate_bollinger_bands(symbol)
        price = get_price(symbol)

        if price is None or upper_band is None or lower_band is None:
            time.sleep(60)  # Sleep for 60 seconds
            continue

        if price > upper_band:
            # Sell logic here
            logging.info(f"Sell signal generated for {symbol} at price {price}")
            send_email("Trading Bot Action", f"Sell signal generated for {symbol} at price {price}")
            # Use your API key and secret key to place a sell order
        elif price < lower_band:
            # Buy logic here
            logging.info(f"Buy signal generated for {symbol} at price {price}")
            send_email("Trading Bot Action", f"Buy signal generated for {symbol} at price {price}")
            # Use your API key and secret key to place a buy order

        time.sleep(60)  # Sleep for 60 seconds

In [ ]:
# Start the trading bot
trade("xmrusdt", "<your-api-key>", "<your-secret-key>")
